In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
import analyze_utils

# https://wandb.ai/jack-morris/emb-inv-1/runs/ebb31d91810c4b62d2b55b5382e8c7ea/logs?workspace=user-jxmorris12
checkpoint_folder = '/home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea'
args_str = '--per_device_train_batch_size 128 --per_device_eval_batch_size 128 --max_seq_length 32 --model_name_or_path t5-base --embedder_model_name gtr_base --num_repeat_tokens 16 --embedder_no_grad True --exp_group_name mar17-baselines --learning_rate 0.0003 --freeze_strategy none --embedder_fake_with_zeros False --use_frozen_embeddings_as_input False --num_train_epochs 24 --max_eval_samples 500 --eval_steps 25000 --warmup_steps 100000 --bf16=1 --use_wandb=1'
trainer = analyze_utils.load_inversion_model_and_trainer(checkpoint_folder, args_str)

trainer.evaluate()

[0] Loading model from checkpoint: /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea/checkpoint-999744
Set train_args.dataloader_num_workers = 4
[1] creating model & stuff


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[2] tokenizing dataset & preprocessing embeddings
[3] initializing trainer
[4] getting ckpnt
[5] loading ckpt /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea/checkpoint-999744


generating from val:   0%|                                                                                           | 0/3 [00:00<?, ?it/s]/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:90: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


to the character of the skull, which are relatively smooth and untutored in the case of infant sutures. Nevertheless, the sutures
to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



individual from the Southern Hemisphere to win the Winter Olympic relay gold medal, and was also part of the Australian Short Track team which won a
individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



the same rights as men, and 75% agreed that they should be protected from discrimination. Among the other 15% are heterosexuals, no work
the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H


{'eval_loss': 1.0522304773330688,
 'eval_bleu_score': 31.552541624779003,
 'eval_accuracy': 0.7384440104166666,
 'eval_perplexity': 2.8640321578446315,
 'eval_runtime': 21.214,
 'eval_samples_per_second': 23.569,
 'eval_steps_per_second': 0.189}

## checking with sampling

In [33]:
batch = next(iter(trainer.get_eval_dataloader()))
true_text = trainer.model.embedder_tokenizer.batch_decode(batch['embedder_input_ids'], skip_special_tokens=True)

In [ ]:
import numpy as np
import torch
import tqdm

torch.set_grad_enabled(False)
gen_kwargs = trainer.gen_kwargs

def sample_text_with_noise(embedder_input_ids: torch.Tensor, embedder_attention_mask: torch.Tensor, alpha: float):
    inputs_embeds, attention_mask = trainer.model.embed_and_project(
        embedder_input_ids=embedder_input_ids.cuda(),
        embedder_attention_mask=embedder_attention_mask.cuda(),
    )
    inputs_embeds += torch.randn(inputs_embeds.shape, device=inputs_embeds.device) * alpha
    return trainer.model.encoder_decoder.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=attention_mask,
        **gen_kwargs,
    )

data = []
for alpha in tqdm.tqdm(np.arange(0, 2.0, 0.1)):
    o = sample_text_with_noise(
        embedder_input_ids=batch["embedder_input_ids"],
        embedder_attention_mask=batch["embedder_attention_mask"],
        alpha=alpha,
    )
    text = trainer.model.embedder_tokenizer.batch_decode(o, skip_special_tokens=True)
    for idx in range(len(text)):
        data.append( (idx, alpha, text[idx]) )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:10<00:00,  1.99it/s]


In [42]:
import pandas as pd
df = pd.DataFrame(data, columns=['eval_idx', 'alpha', 'text'])

In [43]:
df[df['alpha'] == 0.0]

,eval_idx,alpha,text
0,0,0.0,"to the character of the skull, which are relatively smooth and untutored in the case of infant sutures. Nevertheless, the sutures"
1,1,0.0,"individual from the Southern Hemisphere to win the Winter Olympic relay gold medal, and was also part of the Australian Short Track team which won a"
2,2,0.0,"the same rights as men, and 75% agreed that they should be protected from discrimination. Among the other 15% are heterosexuals, no work"
3,3,0.0,"Bruthen, Victoria Bruthen is a small town located between the River Tambo and Brudale on the Atherton In"
4,4,0.0,"Castle Vale Castle Vale is a castle located between Ennisworth and Mortlake, both in the Borough of Edgeford. Currently,"
5,5,0.0,"1865. After her death, Fisk married on August 1, 1867. In 1869, Simona deafened herself away from her sister"
6,6,0.0,"In 1978, he was a member of the Russian Composers' Society. He was commissioned by the Soviet Broadcasting Corporation in"
7,7,0.0,"Zachary Throne Zachary Throne (born September 3, 1970 in Hollywood, California) is an American actor who has appeared in television and on"
8,8,0.0,"the design of a Quaker biddle by the author Owen Biddle, best known as a builder and a ""Jamie"
9,9,0.0,"from Aberdare in the early 19th century. After two years at Aberdare she became principal of Bosford, then head of the Oxford"


In [49]:
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

import random

batch_size = len(batch['input_ids'])
idx = random.randint(0, batch_size)

print(f'Showing idx {idx}')
print(f"Original text: '{true_text[idx]}'")

df[df['eval_idx'] == idx]

Showing idx 7
Original text: 'Zachary Throne Zachary Throne (born April 3, 1967 in Hollywood, California) is an American actor and musician who has appeared in '


,eval_idx,alpha,text
7,7,0.0,"Zachary Throne Zachary Throne (born September 3, 1970 in Hollywood, California) is an American actor who has appeared in television and on"
135,7,0.1,"Zachary Throne Zachary Throne (born August 3, 1970 in Hollywood, California) is an American actor who has appeared in television and on"
263,7,0.2,"Throne Zachary Throne Zachary Corey (born September 3, 1970 in Hollywood, California) is an American actor who has appeared on the"
391,7,0.3,"Zachary Throne Zachary Karl Throne (born August 3, 1960 in London) is an American actor and musician. He has performed on many"
519,7,0.4,"Throne Zachary Throne Zachary (born September 21, 1976 in New York City) is an American actor. He has performed in television and"
647,7,0.5,"Zachary Throne Zachary John Throne (born September 4, 1877 in Montclair, New York) is an American actor who served as"
775,7,0.6,"of the Aristocrats and performed on stage. The Aristocrats of Windsor appointed Limestone as ""live"" on 8 February"
903,7,0.7,"to reign as king of the Trichy. Zachary Cronin, who was born in August, is temporarily trained as an Iron Master in the"
1031,7,0.8,"the release of the release of the Presidential War on Terror, Thomas Throne to remain and assist individuals who reside in August 6, 2018, Thomas Thr"
1159,7,0.9,"of the sacrificial ballot, and Timothy R. Levine, who was born on the following day, moved to stage. Stage action involves"
